In [1]:
import sys

sys.path.append("../")
import os
import math
import numpy as np
import cv2
import pandas as pd
import tikzplotlib
from pedrec.configs.dataset_configs import get_coco_dataset_cfg_default
from pedrec.datasets.coco_dataset import CocoDataset
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.skeleton_pedrec import SKELETON_PEDREC_JOINTS, SKELETON_PEDREC, SKELETON_PEDREC_JOINT
from pedrec.models.constants.skeleton_coco import SKELETON_COCO_JOINTS
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.visualizers.skeleton_visualizer import draw_skeleton
from pedrec.visualizers.skeleton_3d_visualizer import add_skeleton_3d_to_axes
from pedrec.visualizers.visualization_helper_3d import draw_grid_3d, draw_origin_3d
from random import randint
import matplotlib.pyplot as plt
%matplotlib widget


In [13]:
cfg = PedRecNet50Config()

dataset_cfg = get_coco_dataset_cfg_default()
dataset_cfg.use_mebow_orientation = True
dataset_root = "data/datasets/COCO"

dataset = CocoDataset(dataset_root, DatasetType.TRAIN, dataset_cfg, cfg.model.input_size, None)
dataset_length = len(dataset)
print(dataset_length)

loading annotations into memory...
Done (t=4.58s)
creating index...
index created!
149813


In [3]:
fig, ax = plt.subplots(3,3, figsize=(10,10))
fig_3d = plt.figure(figsize=(10,10))

count = 0
for i in range(0, 3):
    for j in range(0, 3):
        entry = dataset[randint(0, dataset_length)]
        # entry = dataset[count + 606]
        model_input, labels = entry
        skeleton = labels["skeleton"]
        skeleton_3d = labels["skeleton_3d"]
        scale_factor = 3
        skeleton_3d[:, :3] *= scale_factor
        skeleton_3d[:, :3] -= (scale_factor / 2)
        center = labels["center"]
        scale = labels["scale"]
        rotation = labels["rotation"]
        is_real_img = labels["is_real_img"]
        img_path = labels["img_path"]
        skeleton[:, 0] *= model_input.shape[1]
        skeleton[:, 1] *= model_input.shape[0]
        visible_joints = np.sum(skeleton[:, 2])
        body_orientation = labels["orientation"][0][1]
        body_orientation *= 2*math.pi
        body_orientation = math.degrees(body_orientation)
        draw_skeleton(model_input, skeleton)
        img = cv2.cvtColor(model_input, cv2.COLOR_BGR2RGB)
        ax[i, j].imshow(img)
        ax[i, j].set_title(f"{count}: {visible_joints} | Bθ​: {body_orientation:.1f}")

        ax_3d = fig_3d.add_subplot(3, 3, count+1, projection='3d')
        draw_grid_3d(ax_3d, lim=1)
        draw_origin_3d(ax_3d)
        add_skeleton_3d_to_axes(ax_3d, skeleton_3d, size=4)
        ax_3d.set_title(f"{count}: {visible_joints} | Bθ​: {body_orientation:.1f}°")

        # print(f"{count}: Model input shape: {model_input.shape}, Min value: {model_input.min()}, max value: {model_input.max()}")
        # print(f"{count}: center: {center}, scale: {scale}, rotation: {rotation}")
        # print(f"{count}: is_real_img: {is_real_img}")
        # print(f"{count}: visible joints: {visible_joints}")
        # print(f"{count}: path: {img_path}")
        # print("------------")
        count += 1
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
dataset_df_path = os.path.join(dataset_root, "COCO_gt_df_train_direct_4.pkl")
df = pd.read_pickle(dataset_df_path)
pd.options.display.max_columns = None
pd.options.display.float_format= '{:.2f}'.format
print(f"Number of entrys: {df.shape[0]}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/datasets/COCO/COCO_gt_df_train_direct_4.pkl'

In [ ]:
filter_skeleton2d = [col for col in df if col.startswith('skeleton2d')]
filter_body_orientation = [col for col in df if col.startswith('body_orientation')]

In [ ]:
# skeleton 2d calculated columns
skeleton2d_xs = [col for col in df if col.startswith('skeleton2d') and col.endswith('_x')]
skeleton2d_ys = [col for col in df if col.startswith('skeleton2d') and col.endswith('_y')]
skeleton2d_visibles = [col for col in df if col.startswith('skeleton2d') and col.endswith('_visible')]
df["skeleton2d_width"] = df[skeleton2d_xs].max(axis=1) - df[skeleton2d_xs].min(axis=1)
df["skeleton2d_height"] = df[skeleton2d_ys].max(axis=1) - df[skeleton2d_ys].min(axis=1)
df["skeleton2d_size"] = np.sqrt(df["skeleton2d_width"]**2 + df["skeleton2d_height"]**2)
df["skeleton2d_visible_joints"] = df[skeleton2d_visibles].sum(axis=1)

In [ ]:
df["body_orientation_phi"] = df["body_orientation_phi"] * math.pi * 2
df["body_orientation_theta"] = df["body_orientation_theta"] * math.pi


In [8]:
df.head(5)

,img_path,img_size_w,img_size_h,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_nose_visible,skeleton2d_nose_supported,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_left_eye_visible,skeleton2d_left_eye_supported,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_right_eye_visible,skeleton2d_right_eye_supported,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_left_ear_visible,skeleton2d_left_ear_supported,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_right_ear_visible,skeleton2d_right_ear_supported,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_left_shoulder_visible,skeleton2d_left_shoulder_supported,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_right_shoulder_visible,skeleton2d_right_shoulder_supported,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_left_elbow_visible,skeleton2d_left_elbow_supported,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_right_elbow_visible,skeleton2d_right_elbow_supported,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_left_wrist_visible,skeleton2d_left_wrist_supported,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_right_wrist_visible,skeleton2d_right_wrist_supported,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_left_hip_visible,skeleton2d_left_hip_supported,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_right_hip_visible,skeleton2d_right_hip_supported,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_left_knee_visible,skeleton2d_left_knee_supported,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_right_knee_visible,skeleton2d_right_knee_supported,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_left_ankle_visible,skeleton2d_left_ankle_supported,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_right_ankle_visible,skeleton2d_right_ankle_supported,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_hip_center_visible,skeleton2d_hip_center_supported,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_spine_center_visible,skeleton2d_spine_center_supported,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_neck_visible,skeleton2d_neck_supported,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_lower_visible,skeleton2d_head_lower_supported,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_head_upper_visible,skeleton2d_head_upper_supported,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_left_foot_end_visible,skeleton2d_left_foot_end_supported,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_right_foot_end_visible,skeleton2d_right_foot_end_supported,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_left_hand_end_visible,skeleton2d_left_hand_end_supported,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d_right_hand_end_score,skeleton2d_right_hand_end_visible,skeleton2d_right_hand_end_supported,body_orientation_theta,body_orientation_phi,body_orientation_score,body_orientation_visible,skeleton2d_width,skeleton2d_height,skeleton2d_size,skeleton2d_visible_joints
0,data/datasets/COCO/train20...,640.00,360.00,366.90,55.57,0.92,1.00,1.00,370.21,48.78,0.89,1.00,1.00,368.97,49.95,0.01,0.00,0.00,384.30,48.16,0.98,1.00,1.00,396.21,50.32,0.13,0.00,0.00,371.3

In [9]:
df.describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))


,img_size_w,img_size_h,skeleton2d_nose_x,skeleton2d_nose_y,skeleton2d_nose_score,skeleton2d_left_eye_x,skeleton2d_left_eye_y,skeleton2d_left_eye_score,skeleton2d_right_eye_x,skeleton2d_right_eye_y,skeleton2d_right_eye_score,skeleton2d_left_ear_x,skeleton2d_left_ear_y,skeleton2d_left_ear_score,skeleton2d_right_ear_x,skeleton2d_right_ear_y,skeleton2d_right_ear_score,skeleton2d_left_shoulder_x,skeleton2d_left_shoulder_y,skeleton2d_left_shoulder_score,skeleton2d_right_shoulder_x,skeleton2d_right_shoulder_y,skeleton2d_right_shoulder_score,skeleton2d_left_elbow_x,skeleton2d_left_elbow_y,skeleton2d_left_elbow_score,skeleton2d_right_elbow_x,skeleton2d_right_elbow_y,skeleton2d_right_elbow_score,skeleton2d_left_wrist_x,skeleton2d_left_wrist_y,skeleton2d_left_wrist_score,skeleton2d_right_wrist_x,skeleton2d_right_wrist_y,skeleton2d_right_wrist_score,skeleton2d_left_hip_x,skeleton2d_left_hip_y,skeleton2d_left_hip_score,skeleton2d_right_hip_x,skeleton2d_right_hip_y,skeleton2d_right_hip_score,skeleton2d_left_knee_x,skeleton2d_left_knee_y,skeleton2d_left_knee_score,skeleton2d_right_knee_x,skeleton2d_right_knee_y,skeleton2d_right_knee_score,skeleton2d_left_ankle_x,skeleton2d_left_ankle_y,skeleton2d_left_ankle_score,skeleton2d_right_ankle_x,skeleton2d_right_ankle_y,skeleton2d_right_ankle_score,skeleton2d_hip_center_x,skeleton2d_hip_center_y,skeleton2d_hip_center_score,skeleton2d_spine_center_x,skeleton2d_spine_center_y,skeleton2d_spine_center_score,skeleton2d_neck_x,skeleton2d_neck_y,skeleton2d_neck_score,skeleton2d_head_lower_x,skeleton2d_head_lower_y,skeleton2d_head_lower_score,skeleton2d_head_upper_x,skeleton2d_head_upper_y,skeleton2d_head_upper_score,skeleton2d_left_foot_end_x,skeleton2d_left_foot_end_y,skeleton2d_left_foot_end_score,skeleton2d_right_foot_end_x,skeleton2d_right_foot_end_y,skeleton2d_right_foot_end_score,skeleton2d_left_hand_end_x,skeleton2d_left_hand_end_y,skeleton2d_left_hand_end_score,skeleton2d_right_hand_end_x,skeleton2d_right_hand_end_y,skeleton2d_right_hand_end_score,body_orientation_theta,body_orientation_phi,body_orientation_score,skeleton2d_width,skeleton2d_height,skeleton2d_size,skeleton2d_visible_joints
count,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813,149813
mean,586.355,479.881,292.531,176.344,0.675979,296.058,170.101,0.588586,289.014,170.142,0.590221,301.062,171.69,0.512442,284.284,172.124,0.515652,306.904,201.115,0.870267,278.221,201.348,0.872983,311.942,239.114,0.689935,273.063,238.603,0.698789,305.872,245.903,0.63335,279.052,244.272,0.641308,302.021,274.968,0.742582,283.19,274.806,0.744286,301.743,295.686,0.570344,282.622,295.191,0.567552,298.628,319.039,0.476967,285.678,320.103,0.474814,293.017,276.179,0.987926,293.339,239.787,0.999793,292.35,195.118,0.997682,292.683,180.965,0.994824,292.527,150.661,0.995909,301.054,317.221,0.985874,287.803,318.494,0.973302,303.665,252.649,0.969546,280.094,248.816,0.964899,1.57085,3.86971,1,107.965,197.019,229.209,19.862
std,87.2193,93.3023,156.41,100.099,0.415205,157.007,100.067,0.441241,157.356,100.218,0.440476,160.9,101.049,0.407863,161.301,100.877,0.408073,164.702,106.126,0.288575,165.396,105.981,0.285968,163.552,113.465,0.382862,164.818,113.4,0.380008,158.684,112.908,0.390559,159.756,112.71,0.389962,162.685,119.709,0.37689,163.062,119.695,0.375703,157.169,121.27,0.449299,158.122,120.68,0.449606,159.114,123.936,0.461049,159.935,124.401,0.460371,161.884,120.077,0.0756394,163.492,115.38,0.0100422,163.535,104.769,0.038094,16

In [10]:
df_description = df.describe()
num_3d_models = 1
num_animations = 1
print("General Report")
print(f"Number of frames: {len(df)}")
print(f"FPS: 30")
print(f"Resolution: 1920x1080")
print(f"Number of 3D models: {num_3d_models}")
print(f"Number of animations {num_animations}")
print("Data & Mean & Std & Min & Max \\\\")
print(f"Skeleton 2D Diameter & ${df_description['skeleton2d_size']['mean']:.2f}px$ & ${df_description['skeleton2d_size']['std']:.2f}px$ & ${df_description['skeleton2d_size']['min']:.2f}px$ & ${df_description['skeleton2d_size']['max']:.2f}px$ \\\\")

General Report
Number of frames: 149813
FPS: 30
Resolution: 1920x1080
Number of 3D models: 1
Number of animations 1
Data & Mean & Std & Min & Max \\
Skeleton 2D Diameter & $229.21px$ & $148.16px$ & $28.42px$ & $1002.18px$ \\


In [15]:
x = []
for _, labels in dataset:
  body_orientation = labels["orientation"][0][1]
  body_orientation *= 2*math.pi
  body_orientation = math.degrees(body_orientation)
  x.append(body_orientation)


In [19]:
# print(x)
# x = np.array(x, dtype=np.float32)
max_val = 0
for i in range (0, 370, 10):
  elements = x[np.logical_and(i <= x, x <= i+10)]
  num_els = len(elements)
  max_val = max_val if max_val > num_els else num_els
  print(f"{i},{num_els}")
print(max_val)

0,11575
1,0
2,0
3,0
4,490
5,918
6,0
7,0
8,0
9,583
10,1203
11,0
12,0
13,0
14,1131
15,484
16,0
17,0
18,0
19,918
20,418
21,0
22,0
23,0
24,0
25,758
26,0
27,0
28,0
29,292
30,640
31,0
32,0
33,0
34,512
35,272
36,0
37,0
38,0
39,642
40,314
41,0
42,0
43,0
44,695
45,695
46,0
47,0
48,0
49,0
50,845
51,0
52,0
53,0
54,451
55,837
56,0
57,0
58,0
59,935
60,458
61,0
62,0
63,0
64,1102
65,1102
66,0
67,0
68,0
69,1252
70,1252
71,0
72,0
73,0
74,1385
75,1385
76,0
77,0
78,0
79,1434
80,730
81,0
82,0
83,0
84,1383
85,1383
86,0
87,0
88,0
89,1033
90,1033
91,0
92,0
93,0
94,1295
95,1295
96,0
97,0
98,0
99,0
100,1487
101,0
102,0
103,0
104,665
105,1371
106,0
107,0
108,0
109,1188
110,1188
111,0
112,0
113,0
114,1070
115,1070
116,0
117,0
118,0
119,918
120,918
121,0
122,0
123,0
124,871
125,0
126,0
127,0
128,0
129,835
130,835
131,0
132,0
133,0
134,688
135,688
136,0
137,0
138,0
139,601
140,601
141,0
142,0
143,0
144,555
145,555
146,0
147,0
148,0
149,682
150,682
151,0
152,0
153,0
154,787
155,787
156,0
157,0
158,0
159,935
160,935

In [12]:
fig = plt.figure()
body_phi_deg = df['body_orientation_phi'] * (180 / math.pi)
body_phi_deg.hist(bins=50)
xmin, xmax, ymin, ymax = plt.axis()
tikzplotlib.save("/home/dennis/Downloads/coco_body_orientation_phi.tex", extra_axis_parameters={
  'width=0.7\\textwidth',
  'font=\\footnotesize',
  'title={Distribution of body $\\theta$ orientations}',
  'xlabel={degrees ($°$)}',
  'ylabel={Number of samples}',
  'enlarge x limits=0.001',
  'enlarge y limits=0.001',
  'xmin=0',
  'xmax=360',
})

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig = plt.figure()
df['skeleton2d_width'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton2d_height'].hist(bins=50)

In [ ]:
fig = plt.figure()
df['skeleton2d_size'].hist(bins=50)

In [8]:
fig = plt.figure()
df['body_orientation_phi'].hist(bins=50)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [ ]:
dataset_df_path = os.path.join(dataset_root, "COCO_pred_df_val_direct_4.pkl")
df = pd.read_pickle(dataset_df_path)
pd.options.display.max_columns = None
pd.options.display.float_format= '{:.2f}'.format
print(f"Number of entrys: {df.shape[0]}")


In [ ]:
filter_skeleton2d = [col for col in df if col.startswith('skeleton2d')]
skeleton_2d = df[skeleton2d_xs].to_numpy(dtype=np.float32).reshape(len(df), len(SKELETON_PEDREC_JOINTS), 5)
coco_out = []
for joint in SKELETON_COCO_JOINTS:
  coco_out.append(skeleton_2d[joint.value, :, :])
a = 1


